In [1]:
#!pip install findspark
#!pip install mpld3
#!pip install pymongo
#!pip install sodapy
#!pip install sqlalchemy
#!pip install pyspark

In [1]:
import numpy as np
import pandas as pd
import random
import requests
import json
import plotly.graph_objects as go
import plotly.offline as plotly
import findspark
import psycopg2
import os
import matplotlib.pyplot as plt
import mpld3
import pymongo
from pymongo import MongoClient
from sodapy import Socrata
from pyspark.sql import SparkSession
from flask import Flask, request, render_template, jsonify
from sqlalchemy import create_engine
import plotly.graph_objects as go
import plotly.offline as plotly

# Import Data

In [2]:
url = "https://data.cityofnewyork.us/resource/mwzb-yiwb.json?$limit=1000000"

In [3]:
r = requests.get(url)
nyc_expense_data_json = r.json()

In [4]:
nyc_expense_df = pd.read_csv('/Users/hollyshi/Desktop/Managing Data - Final Project /Expense_Budget-v1.csv')

/var/folders/v3/2l10gjx501g4hvjj3xtnw6vc0000gn/T/ipykernel_4851/2968748218.py:1: DtypeWarning:

Columns (2,13) have mixed types. Specify dtype option on import or set low_memory=False.



In [5]:
print(nyc_expense_df.head())

   Publication Date  Fiscal Year Agency Number        Agency Name  \
0          20220614         2023            56  POLICE DEPARTMENT   
1          20220614         2023            56  POLICE DEPARTMENT   
2          20220614         2023            56  POLICE DEPARTMENT   
3          20220614         2023            56  POLICE DEPARTMENT   
4          20220614         2023            56  POLICE DEPARTMENT   

   Unit Appropriation Number Unit Appropriation Name Budget Code Number  \
0                          7     TRAFFIC ENFORCEMENT               7406   
1                          7     TRAFFIC ENFORCEMENT               7410   
2                          7     TRAFFIC ENFORCEMENT               7410   
3                          7     TRAFFIC ENFORCEMENT               7410   
4                          7     TRAFFIC ENFORCEMENT               7410   

  Budget Code Name  Object Class Number     Object Class Name  ...  \
0     PROJECT HELP                    6       FRINGE BENEFITS  .

In [6]:
nyc_expense_data_json[0]

{'publication_date': '20230630',
 'fiscal_year': '2024',
 'agency_number': '002',
 'agency_name': 'MAYORALTY',
 'unit_appropriation_number': '091',
 'unit_appropriation_name': "MAYOR'S OFFICE OF CONTRACT SERVICES-OTPS",
 'budget_code_number': '0901',
 'budget_code_name': 'Technology Strategy',
 'object_class_number': '60',
 'object_class_name': 'CONTRACTUAL SERVICES',
 'object_code': '671',
 'object_code_name': 'TRAINING PRGM CITY EMPLOYEES',
 'personal_service_other_than_personal_service_indicator': 'O',
 'financial_plan_savings_flag': 'N',
 'adopted_budget_amount': '0',
 'current_modified_budget_amount': '8700',
 'financial_plan_amount': '0',
 'adopted_budget_position': '0',
 'current_modified_budget_position': '0',
 'financial_plan_position': '0',
 'adopted_budget_number_of_contracts': '0',
 'current_modified_budget_number_of_contracts': '1',
 'financial_plan_number_of_contracts': '0'}

In [7]:
nyc_expense_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 809652 entries, 0 to 809651
Data columns (total 26 columns):
 #   Column                                                  Non-Null Count   Dtype  
---  ------                                                  --------------   -----  
 0   Publication Date                                        809652 non-null  int64  
 1   Fiscal Year                                             809652 non-null  int64  
 2   Agency Number                                           809652 non-null  object 
 3   Agency Name                                             809652 non-null  object 
 4   Unit Appropriation Number                               809652 non-null  int64  
 5   Unit Appropriation Name                                 809652 non-null  object 
 6   Budget Code Number                                      809652 non-null  object 
 7   Budget Code Name                                        809652 non-null  object 
 8   Object Class Number     

In [8]:
nyc_expense_df.head()

Publication Date  Fiscal Year Agency Number        Agency Name  \
0          20220614         2023            56  POLICE DEPARTMENT   
1          20220614         2023            56  POLICE DEPARTMENT   
2          20220614         2023            56  POLICE DEPARTMENT   
3          20220614         2023            56  POLICE DEPARTMENT   
4          20220614         2023            56  POLICE DEPARTMENT   

   Unit Appropriation Number Unit Appropriation Name Budget Code Number  \
0                          7     TRAFFIC ENFORCEMENT               7406   
1                          7     TRAFFIC ENFORCEMENT               7410   
2                          7     TRAFFIC ENFORCEMENT               7410   
3                          7     TRAFFIC ENFORCEMENT               7410   
4                          7     TRAFFIC ENFORCEMENT               7410   

  Budget Code Name  Object Class Number     Object Class Name  ...  \
0     PROJECT HELP                    6       FRINGE BENEFITS  ...   
1    VIOLATION TOW                    1    FULL TIME SALARIED  ...   
2    VIOLATION TOW                    4  ADDITIONAL GROSS PAY  ...   
3    VIOLATION TOW                    4  ADDITIONAL GROSS PAY  ...   
4    VIOLATION TOW                    6       FRINGE BENEFITS  ...   

  Financial Plan Savings Flag Adopted Budget Amount  \
0                           N                     0   
1                           N               7621551   
2                           N                  3888   
3                           N                250000   
4                           N                  2732   

   Current Modified Budget Amount Financial Plan Amount  \
0                          769687                     0   
1                         7621551               7621551   
2                            3888                  3888   
3                          250000                250000   
4                            2732                  2732   

  Adopted Budget Position Current Modified Budget Position  \
0                       0                                0   
1                     160                              160   
2                       0                                0   
3                       0                                0   
4                       0                                0   

  Financial Plan Position  Adopted Budget - Number of Contracts  \
0                       0                                     0   
1                     160                                     0   
2                       0                                     0   
3                       0                                     0   
4                       0                                     0   

   Current Modified Budget - Number of Contracts  \
0                                              0   
1                                              0   
2                                              0   
3                                              0   
4                                              0   

   Financial Plan - Number of Contracts  
0                                     0  
1                                     0  
2                                     0  
3                                     0  
4                                     0  

[5 rows x 26 columns]

In [9]:
for dict_obj in nyc_expense_data_json:
    keys_to_remove = []
    for key in dict_obj.keys():
        if key.startswith(':@computed_region'):
            keys_to_remove.append(key)
    for key in keys_to_remove:
        dict_obj.pop(key)


In [10]:
nyc_expense_data_json[10000]

{'publication_date': '20230630',
 'fiscal_year': '2024',
 'agency_number': '040',
 'agency_name': 'DEPARTMENT OF EDUCATION',
 'unit_appropriation_number': '439',
 'unit_appropriation_name': 'SCHOOL FOOD SERVICES - PS',
 'budget_code_number': '1229',
 'budget_code_name': 'DIRECT FIELD OPERATIONS',
 'object_class_number': '03',
 'object_class_name': 'UNSALARIED',
 'object_code': '031',
 'object_code_name': 'UNSALARIED',
 'personal_service_other_than_personal_service_indicator': 'P',
 'financial_plan_savings_flag': 'N',
 'adopted_budget_amount': '160697052',
 'current_modified_budget_amount': '177882415',
 'financial_plan_amount': '172664763',
 'adopted_budget_position': '0',
 'current_modified_budget_position': '0',
 'financial_plan_position': '0',
 'adopted_budget_number_of_contracts': '0',
 'current_modified_budget_number_of_contracts': '0',
 'financial_plan_number_of_contracts': '0'}

# MangoDB Connection & Data Analysis

In [11]:
client = MongoClient("localhost", 27017)
db = client.ManagingData_database
collection = db.expense_collection

In [12]:
result = collection.delete_many({})
print(f"Number of documents deleted: {result.deleted_count}")
collection.insert_many(nyc_expense_data_json)

Number of documents deleted: 11335128


# Top 10 Agencies with Highest Adoped Expense Budget 

In [13]:
pipeline = [
    {
        '$group': {
            '_id': '$agency_name',
            'total_budget_expense': {'$sum': {'$toDouble': '$adopted_budget_amount'}}
        }
    },
    {
        '$sort': {
            'total_budget_expense': -1 
        }
    },
    {
        '$limit': 10  
    }
]

result = collection.aggregate(pipeline)

agency_names = []
budget_expenses = []
for doc in result:
    agency_names.append(doc['_id'])
    budget_expenses.append(doc['total_budget_expense'])

if not budget_expenses or all(expense == 0 for expense in budget_expenses):
    print("No non-zero budget expenses data to plot.")
else:
    fig = go.Figure([go.Bar(x=agency_names, y=budget_expenses)])

    fig.update_layout(
        title='Top 10 Agencies by Budget Expense',
        xaxis_title='Agency Name',
        yaxis_title='Aggregated Budget Expense',
        yaxis_tickprefix="$",  
        hovermode='x', 
    )

    y_range_padding = 0.1
    fig.update_yaxes(range=[0, max(budget_expenses, default=1) * (1 + y_range_padding)])

    fig.show()


# Plot for Top 10 Agencies by Highest Expense-to-Planning Ratio

In [14]:
pipeline = [
    {
        '$group': {
            '_id': '$agency_name',
            'total_budget_expense': {'$sum': {'$toDouble': '$adopted_budget_amount'}},
            'total_financial_planning': {'$sum': {'$toDouble': '$financial_plan_amount'}}
        }
    },
    {
        '$match': {
            'total_financial_planning': {'$ne': 0}  
        }
    },
    {
        '$sort': {
            'total_budget_expense': -1 
        }
    },
    {
        '$limit': 10
    },
    {
        '$project': {
            'agency_name': '$_id',
            'budget_expense': '$total_budget_expense',
            'financial_planning': '$total_financial_planning',
            'expense_to_planning_ratio': {'$divide': ['$total_budget_expense', '$total_financial_planning']}
        }
    }
]

result = collection.aggregate(pipeline)


In [15]:
agency_names = []
expense_to_planning_ratios = []
for doc in result:
    agency_names.append(doc['agency_name'])
    expense_to_planning_ratios.append(doc['expense_to_planning_ratio'])

if not expense_to_planning_ratios or all(ratio == 0 for ratio in expense_to_planning_ratios):
    print("No non-zero expense-to-planning ratio data to plot.")
else:
    fig = go.Figure([go.Bar(x=agency_names, y=expense_to_planning_ratios)])

    fig.update_layout(
        title='Top 10 Agencies by Highest Expense-to-Planning Ratio',
        xaxis_title='Agency Name',
        yaxis_title='Expense-to-Planning Ratio',
        hovermode='x', 
    )

    y_range_padding = 0.1
    fig.update_yaxes(range=[0, max(expense_to_planning_ratios, default=1) * (1 + y_range_padding)])

    fig.show()



# Plot for Top 10 Agencies by Lowest Expense-to-Planning Ratio

In [20]:
pipeline = [
    {
        '$group': {
            '_id': '$agency_name',
            'total_budget_expense': {'$sum': {'$toDouble': '$adopted_budget_amount'}},
            'total_financial_planning': {'$sum': {'$toDouble': '$financial_plan_amount'}}
        }
    },
    {
        '$match': {
            'total_financial_planning': {'$ne': 0}  
        }
    },
    {
        '$project': {
            'agency_name': '$_id',
            'budget_expense': '$total_budget_expense',
            'financial_planning': '$total_financial_planning',
            'expense_to_planning_ratio': {'$divide': ['$total_budget_expense', '$total_financial_planning']}
        }
    },
    {
        '$sort': {
            'expense_to_planning_ratio': 1   
        }
    },
    {
        '$limit': 10
    }
]

result = collection.aggregate(pipeline)


In [21]:
agency_names = []
expense_to_planning_ratios = []
for doc in result:
    agency_names.append(doc['agency_name'])
    expense_to_planning_ratios.append(doc['expense_to_planning_ratio'])

if not expense_to_planning_ratios or all(ratio == 0 for ratio in expense_to_planning_ratios):
    print("No non-zero expense-to-planning ratio data to plot.")
else:
    fig = go.Figure([go.Bar(x=agency_names, y=expense_to_planning_ratios)])

    fig.update_layout(
        title='Top 10 Agencies by Lowest Expense-to-Planning Ratio',
        xaxis_title='Agency Name',
        yaxis_title='Expense-to-Planning Ratio',
        hovermode='x', 
    )

    y_range_padding = 0.1
    fig.update_yaxes(range=[0, max(expense_to_planning_ratios, default=1) * (1 + y_range_padding)])

    fig.show()


In [22]:
agency_name_to_check = 'ENERGY ADJUSTMENT'

pipeline = [
    {
        '$match': {
            'agency_name': agency_name_to_check
        }
    },
    {
        '$group': {
            '_id': '$agency_name',
            'total_budget_expense': {'$sum': {'$toDouble': '$adopted_budget_amount'}},
            'total_financial_planning': {'$sum': {'$toDouble': '$financial_plan_amount'}}
        }
    }
]

result = collection.aggregate(pipeline)

agency_data = next(result, None)

if agency_data:
    total_budget_expense = agency_data.get('total_budget_expense', 0)
    total_financial_planning = agency_data.get('total_financial_planning', 0)

    print(f"Agency Name: {agency_name_to_check}")
    print(f"Total Adopted Expense Amount: {total_budget_expense}")
    print(f"Total Financial Plan Amount: {total_financial_planning}")
else:
    print(f"No data found for the agency: {agency_name_to_check}")

Agency Name: ENERGY ADJUSTMENT
Total Adopted Expense Amount: 0.0
Total Financial Plan Amount: 204829418.0


In [23]:
agency_name_to_check = 'OFFICE OF RACIAL EQUITY'

pipeline = [
    {
        '$match': {
            'agency_name': agency_name_to_check
        }
    },
    {
        '$group': {
            '_id': '$agency_name',
            'total_budget_expense': {'$sum': {'$toDouble': '$adopted_budget_amount'}},
            'total_financial_planning': {'$sum': {'$toDouble': '$financial_plan_amount'}}
        }
    }
]

result = collection.aggregate(pipeline)

agency_data = next(result, None)

if agency_data:
    total_budget_expense = agency_data.get('total_budget_expense', 0)
    total_financial_planning = agency_data.get('total_financial_planning', 0)

    print(f"Agency Name: {agency_name_to_check}")
    print(f"Total Adopted Expense Amount: {total_budget_expense}")
    print(f"Total Financial Plan Amount: {total_financial_planning}")
else:
    print(f"No data found for the agency: {agency_name_to_check}")

Agency Name: OFFICE OF RACIAL EQUITY
Total Adopted Expense Amount: 0.0
Total Financial Plan Amount: 8011258.0


In [24]:
agency_name_to_check = 'CITYWIDE SAVINGS INITIATIVES'

pipeline = [
    {
        '$match': {
            'agency_name': agency_name_to_check
        }
    },
    {
        '$group': {
            '_id': '$agency_name',
            'total_budget_expense': {'$sum': {'$toDouble': '$adopted_budget_amount'}},
            'total_financial_planning': {'$sum': {'$toDouble': '$financial_plan_amount'}}
        }
    }
]

result = collection.aggregate(pipeline)

# Extract data for the agency
agency_data = next(result, None)

if agency_data:
    total_budget_expense = agency_data.get('total_budget_expense', 0)
    total_financial_planning = agency_data.get('total_financial_planning', 0)

    print(f"Agency Name: {agency_name_to_check}")
    print(f"Total Adopted Expense Amount: {total_budget_expense}")
    print(f"Total Financial Plan Amount: {total_financial_planning}")
else:
    print(f"No data found for the agency: {agency_name_to_check}")

Agency Name: CITYWIDE SAVINGS INITIATIVES
Total Adopted Expense Amount: 0.0
Total Financial Plan Amount: -940445077.0


In [25]:
agency_name_to_check = 'CRIMINAL JUSTICE COORDINATOR'

pipeline = [
    {
        '$match': {
            'agency_name': agency_name_to_check
        }
    },
    {
        '$group': {
            '_id': '$agency_name',
            'total_budget_expense': {'$sum': {'$toDouble': '$adopted_budget_amount'}},
            'total_financial_planning': {'$sum': {'$toDouble': '$financial_plan_amount'}}
        }
    }
]

result = collection.aggregate(pipeline)

# Extract data for the agency
agency_data = next(result, None)

if agency_data:
    total_budget_expense = agency_data.get('total_budget_expense', 0)
    total_financial_planning = agency_data.get('total_financial_planning', 0)

    print(f"Agency Name: {agency_name_to_check}")
    print(f"Total Adopted Expense Amount: {total_budget_expense}")
    print(f"Total Financial Plan Amount: {total_financial_planning}")
else:
    print(f"No data found for the agency: {agency_name_to_check}")

Agency Name: CRIMINAL JUSTICE COORDINATOR
Total Adopted Expense Amount: 0.0
Total Financial Plan Amount: 44947138.0


In [26]:
agency_name_to_check = 'LEASE ADJUSTMENT'

pipeline = [
    {
        '$match': {
            'agency_name': agency_name_to_check
        }
    },
    {
        '$group': {
            '_id': '$agency_name',
            'total_budget_expense': {'$sum': {'$toDouble': '$adopted_budget_amount'}},
            'total_financial_planning': {'$sum': {'$toDouble': '$financial_plan_amount'}}
        }
    }
]

result = collection.aggregate(pipeline)

# Extract data for the agency
agency_data = next(result, None)

if agency_data:
    total_budget_expense = agency_data.get('total_budget_expense', 0)
    total_financial_planning = agency_data.get('total_financial_planning', 0)

    print(f"Agency Name: {agency_name_to_check}")
    print(f"Total Adopted Expense Amount: {total_budget_expense}")
    print(f"Total Financial Plan Amount: {total_financial_planning}")
else:
    print(f"No data found for the agency: {agency_name_to_check}")

Agency Name: LEASE ADJUSTMENT
Total Adopted Expense Amount: 0.0
Total Financial Plan Amount: 265139406.0


In [27]:
agency_name_to_check = 'COMMISSION ON RACIAL EQUITY'

pipeline = [
    {
        '$match': {
            'agency_name': agency_name_to_check
        }
    },
    {
        '$group': {
            '_id': '$agency_name',
            'total_budget_expense': {'$sum': {'$toDouble': '$adopted_budget_amount'}},
            'total_financial_planning': {'$sum': {'$toDouble': '$financial_plan_amount'}}
        }
    }
]

result = collection.aggregate(pipeline)

# Extract data for the agency
agency_data = next(result, None)

if agency_data:
    total_budget_expense = agency_data.get('total_budget_expense', 0)
    total_financial_planning = agency_data.get('total_financial_planning', 0)

    print(f"Agency Name: {agency_name_to_check}")
    print(f"Total Adopted Expense Amount: {total_budget_expense}")
    print(f"Total Financial Plan Amount: {total_financial_planning}")
else:
    print(f"No data found for the agency: {agency_name_to_check}")

Agency Name: COMMISSION ON RACIAL EQUITY
Total Adopted Expense Amount: 0.0
Total Financial Plan Amount: 2600000.0


# Plot for agencies without growth 

In [28]:
fig = go.Figure()
df = pd.DataFrame(nyc_expense_data_json)
agency_names = df['agency_name'].unique()
df['adopted_budget_amount'] = df['adopted_budget_amount'].astype(int)
colors = ['blue', 'green', 'red', 'orange', 'purple']

def calculate_budget_change_rate(adopted_budget_amount_series):
    return adopted_budget_amount_series.diff().fillna(0)

fig = go.Figure()

agency_names = df['agency_name'].unique()

for i, agency_name in enumerate(agency_names):
    df_agency = df[df['agency_name'] == agency_name].copy()
    if len(df_agency) > 1:
        df_agency['budget_change_rate'] = calculate_budget_change_rate(df_agency['adopted_budget_amount'])
        std_budget_change_rate = df_agency['budget_change_rate'].std()
        steadiness_threshold = 100
        if std_budget_change_rate < steadiness_threshold:
            fig.add_trace(go.Histogram(x=df_agency['fiscal_year'], y=df_agency['adopted_budget_amount'],
                                       name=agency_name, marker_color=colors[i % len(colors)]))

if len(fig.data) == 0:
    print("No agencies with steady growth rate found.")
else:
    fig.update_layout(
        title='Total Expense by Agency Name and Year (Histogram)',
        xaxis_title='Year',
        yaxis_title='Total Expense',
        barmode='group'
    )

    fig.show()



0             0
1             0
2             0
3             0
4             0
          ...  
809647     1000
809648      356
809649     5000
809650     5000
809651    12324
Name: adopted_budget_amount, Length: 809652, dtype: int64
